In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10737, done.
remote: Total 10737 (delta 0), reused 0 (delta 0), pack-reused 10737
Receiving objects: 100% (10737/10737), 10.83 MiB | 21.91 MiB/s, done.
Resolving deltas: 100% (7425/7425), done.


In [ ]:
import torch
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)
#model.cuda()
model.cpu()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-1-28 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)



  0%|          | 0.00/147M [00:00<?, ?B/s]

Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      

In [ ]:
#Fetch Video Properties
cap = cv2.VideoCapture('Cyclists ignoring pedestrians at zebra crossing.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)


out_stream = cv2.VideoWriter(
    'Output.mp4',
    cv2.VideoWriter_fourcc('X','V','I','D'),
    5,
    (int(width),int(height)))


sec = 0
frameRate = 0.33

while True:

  # Capture Frame-by-Frame
  cap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  ret, img = cap.read()
  sec = sec + frameRate



  if ret == False:
    break

  # Get Frame Dimentions
  height, width, channels = img.shape
  results = model([img])

  class_ids = []
  confidences = []
  boxes = []

  outs = results.pandas().xyxy[0][results.pandas().xyxy[0]['confidence'] > 0.5]
  outs = pd.DataFrame(outs[outs['name'] == 'person'])

  for index, out in outs.iterrows():

    # Get Center, Height and Width of the Box
    w = out['xmax'] - out['xmin']
    h = out['ymax'] - out['ymin']

    # Topleft Co-ordinates
    x = out['xmin']
    y = out['ymin']

    boxes.append([int(x), int(y), int(w), int(h)])

  persons = []

  for box in boxes:
    (x, y, w, h) = box
    persons.append([x,y,w,h])

  # Calculate Coordinates of People Detected
  person_coordinates = []

  for p in range(len(persons)):
    person_coordinates.append((persons[p][0]+int(persons[p][2]/2),persons[p][1]+int(persons[p][3]/2)))


  # Count
  person_count = len(persons)
  xpos = 15


  # Show Monitoring Status in a Black Box at the Top
  cv2.rectangle(img,(0,0),(width,50),(0,0,0),-1)
  cv2.rectangle(img,(1,1),(width-1,50),(255,255,255),2)

  string = "Total People = "+str(person_count)
  cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

  # Put Bounding Boxes on People in the Frame
  for p in range(person_count):

    a,b,c,d = persons[p]
    cv2.rectangle(img, (a, b), (a + c, b + d), (0,255,0), 2)
    cv2.putText(img , 'person' , (a,b) , cv2.FONT_HERSHEY_SIMPLEX , 1 ,(255,255,255) , 1 , 2)

#--------------------------------------------------------------


  confidences_c = []
  boxes_c = []


  outs_c = results.pandas().xyxy[0][results.pandas().xyxy[0]['confidence'] > 0.5]
  outs_c = pd.DataFrame(outs_c[outs_c['name'] == 'car'])

  for index, out in outs_c.iterrows():

    # Get Center, Height and Width of the Box
    w = out['xmax'] - out['xmin']
    h = out['ymax'] - out['ymin']

    # Topleft Co-ordinates
    x = out['xmin']
    y = out['ymin']

    boxes_c.append([int(x), int(y), int(w), int(h)])

  cars = []

  for box in boxes_c:
    (x, y, w, h) = box
    cars.append([x,y,w,h])

  cars_coordinates = []

  for p in range(len(cars)):
    cars_coordinates.append((cars[p][0]+int(cars[p][2]/2),cars[p][1]+int(cars[p][3]/2)))


  # Count
  car_count = len(cars)

  string = "Total Cars = "+str(car_count)
  cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]





  # Put Bounding Boxes on People in the Frame
  for p in range(car_count):

    a,b,c,d = cars[p]
    cv2.rectangle(img, (a, b), (a + c, b + d), (0,255,0), 2)
    cv2.putText(img , 'car' , (a,b) , cv2.FONT_HERSHEY_SIMPLEX , 1 ,(255,255,255) , 1 , 2)





  # Write Frame to the Output File
  out_stream.write(img)

  # Exit on Pressing Q Key
  if cv2.waitKey(25) & 0xFF == ord('q'):
      break

# Release Streams
out_stream.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
!nvidia-smi -L


GPU 0: Tesla K80 (UUID: GPU-bd958591-4c30-d5be-0fe8-f7a9c5cd0f1d)
